# **Algoritmos Genéricos**

<p style="font-size: smaller; text-align: right;">João Dias & Rafael Rodrigues</p>



## **Objetivo**

Aplicar um Algoritmo Genético (AG) para encontrar o valor máximo da função 
𝑓(𝑥)=𝑥2−3𝑥+4f(x)=x2−3x+4 no intervalo 𝑋=[−10,+10].



## **Estrutura do Notebook**

1. **Introdução**  
   Explicação sobre os algoritmos genéticos e sua aplicação na otimização de funções.

2. **Representação e População Inicial**  
   Codificação dos indivíduos e geração da população inicial.

3. **Função Objetivo**  
   Implementação da função 𝑓(𝑥).


4. **Operadores Genéticos**  
   Implementação das operações de seleção, crossover e mutação.

5. **Execução do Algoritmo**  
   Evolução ao longo de várias gerações e visualização dos resultados.

6. **Conclusão**
Análise dos resultados e avaliação da eficácia do algoritmo.



## **Introdução**

Os Algoritmos Genéticos (AGs) são meta-heurísticas inspiradas na seleção natural e evolução biológica. Eles utilizam uma população de soluções candidatas que evoluem ao longo de gerações por meio de:

- **Seleção:** Escolha dos melhores indivíduos com base em sua aptidão.
- **Crossover:** Combinação de características entre indivíduos selecionados.
- **Mutação:** Introdução de pequenas variações para manter a diversidade.

Neste notebook, aplicaremos um AG para encontrar a solução que maximiza a função 𝑓(𝑥)no intervalo definido, utilizando:
- População inicial de *4 indivíduos*.
- Taxas de *crossover (70%)* e *mutação (1%)*.
- *Seleção por torneio* como método de escolha.
- Avaliação ao longo de *5 gerações*, podendo ser estendido para *20 gerações*.




---

## **Representação dos indivíduos e População inicial**

Para implementar o Algoritmo Genético (AG), precisamos codificar os valores de 𝑥 como vetores binários. Vamos dividir essa etapa em três partes:

1. *Codificação dos Indivíduos:* Representação de valores de 𝑥 em formato binário.
2. *Decodificação:* Conversão do vetor binário de volta para valores reais no intervalo [−10,10].
3. *Geração da População Inicial:* Criação de uma população inicial de 4 indivíduos.




---

## **1. Codificação dos Indivíduos**

Para representar 𝑥 como um vetor binário, definiremos um número de bits adequado para a precisão desejada.

Vamos usar 8 bits para representar o intervalo [−10,10] Isso nos permite discretizar o intervalo em 2 elevado a 8 = 2562 valores possíveis.
Fórmula de Decodificação: 𝑥 = min + (valor_decimal/(2 elevado a 𝑛) − 1) ⋅ (max − min)

Onde:

- 𝑛 é o número de bits (8).
- valor_decimal é o valor do vetor binário convertido para decimal.
- min e max são os limites do intervalo (−10 e +10).



---

## **2. Implementação da Codificação e Decodificação**

Abaixo, implementaremos as funções de codificação e decodificação:




In [ ]:
import numpy as np

# Parâmetros do problema
NUM_BITS = 8  # Número de bits para representar x
MIN_X = -10   # Limite inferior
MAX_X = 10    # Limite superior

# Função para gerar um indivíduo aleatório (binário)
def generate_individual(num_bits=NUM_BITS):
    return np.random.randint(0, 2, size=num_bits)  # Vetor binário aleatório

# Função para decodificar um vetor binário para um valor real
def decode_individual(binary_vector, min_x=MIN_X, max_x=MAX_X):
    decimal_value = int("".join(map(str, binary_vector)), 2)  # Converte binário para decimal
    x = min_x + (decimal_value / (2**len(binary_vector) - 1)) * (max_x - min_x)
    return x


---

## **3. Geração da População Inicial**

Agora, criaremos a população inicial de 4 indivíduos.




In [ ]:
# Função para gerar a população inicial
def generate_initial_population(pop_size=4, num_bits=NUM_BITS):
    population = [generate_individual(num_bits) for _ in range(pop_size)]
    return population

# Gerar a população inicial
population = generate_initial_population()

# Exibir a população inicial e seus valores decodificados
print("População Inicial:")
for i, individual in enumerate(population):
    x_value = decode_individual(individual)
    print(f"Indivíduo {i+1}: Binário = {individual}, Valor de x = {x_value:.4f}")


---

## **Função de Avaliação**

A função fitness mede o quão “bom” é um indivíduo em resolver o problema. Para este trabalho, o valor de f(x) será a medida de fitness.

Entrada: A população atual (representada em binário).
Decodificação: Converter o vetor binário para valores de x dentro do intervalo [−10,+10].
Cálculo do Fitness: Avaliar f(x) para cada indivíduo.





In [ ]:
def binary_to_decimal(binary, lower_bound, upper_bound):
    """
    Converte um vetor binário para um valor decimal no intervalo [lower_bound, upper_bound].
    """
    decimal = int("".join(map(str, binary)), 2)
    max_value = (2 ** len(binary)) - 1
    scaled_value = lower_bound + (decimal / max_value) * (upper_bound - lower_bound)
    return scaled_value

def fitness_function(population, lower_bound=-10, upper_bound=10):
    """
    Calcula o fitness de cada indivíduo na população.
    """
    fitness_values = []
    for individual in population:
        x = binary_to_decimal(individual, lower_bound, upper_bound)
        fitness = x**2 - 3*x + 4  # Função f(x) = x^2 - 3x + 4
        fitness_values.append(fitness)
    return fitness_values


Explicação:

- binary_to_decimal: Converte um vetor binário em um valor decimal escalado para o intervalo desejado ([−10,+10]).

- fitness_function: Calcula o fitness de cada indivíduo da população ao avaliar f(x).

## **Testando a Função**

Para testar, usaremos a população inicial criada anteriormente.


In [ ]:
# Testando a função fitness
fitness_values = fitness_function(population)
print("Valores de Fitness para a População Inicial:")
for i, fitness in enumerate(fitness_values):
    print(f"Indivíduo {i+1}: Fitness = {fitness:.4f}")


Saída Esperada: Uma lista com os valores de fitness calculados para cada indivíduo.

---

## **Seleção por Torneio**

A seleção por torneio é um método que seleciona os indivíduos com base no valor do fitness. O processo consiste em:

- Selecionar aleatoriamente um grupo de indivíduos da população (tamanho do torneio).
- Comparar os valores de fitness dos indivíduos selecionados.
- Escolher o indivíduo com o melhor fitness para reprodução.
- Esse processo ajuda a garantir que os melhores indivíduos têm maior chance de reproduzir, mas ainda mantém alguma variabilidade devido à seleção aleatória.




In [ ]:
import random

def tournament_selection(population, fitness_values, tournament_size=2):
    """
    Seleção por torneio: seleciona um indivíduo da população com base no fitness.
    """
    selected = []
    for _ in range(len(population)):
        # Escolher aleatoriamente 'tournament_size' indivíduos
        participants = random.sample(list(enumerate(fitness_values)), tournament_size)
        # Encontrar o participante com maior fitness
        winner = max(participants, key=lambda x: x[1])
        selected.append(population[winner[0]])  # Adiciona o indivíduo vencedor
    return selected


Explicação:
- random.sample: Seleciona aleatoriamente tournament_size indivíduos e seus valores de fitness.
- max: Seleciona o indivíduo com o maior fitness entre os participantes.
- Retorno: Retorna uma nova população formada pelos vencedores do torneio.

## **Testando a Seleção por Torneio**

In [ ]:
# Executando a seleção por torneio
selected_population = tournament_selection(population, fitness_values, tournament_size=2)

print("População Selecionada (Torneio):")
for i, individual in enumerate(selected_population):
    print(f"Indivíduo {i+1}: {individual}")


Saída Esperada: A nova população selecionada, com indivíduos que têm maior fitness em relação aos outros participantes do torneio.

---

## **Crossover (Recombinação)**

O crossover combina informações genéticas de dois pais para gerar dois novos filhos. Ele ocorre com uma taxa de crossover definida (geralmente 70%). O processo é:

- Selecionar aleatoriamente dois indivíduos da população.
- Escolher um ponto de corte (posição) no vetor binário.
- Trocar os segmentos entre os dois indivíduos a partir do ponto de corte.
- Adicionar os filhos gerados à nova população.

In [ ]:
def crossover(parent1, parent2):
    """
    Aplica crossover de um ponto entre dois pais.
    """
    point = random.randint(1, len(parent1) - 1)  # Ponto de corte
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def apply_crossover(population, crossover_rate=0.7):
    """
    Aplica crossover na população com uma taxa de crossover.
    """
    new_population = []
    for i in range(0, len(population), 2):
        parent1 = population[i]
        parent2 = population[i + 1 if i + 1 < len(population) else 0]  # Garante paridade
        if random.random() < crossover_rate:
            child1, child2 = crossover(parent1, parent2)
        else:
            child1, child2 = parent1, parent2  # Sem crossover
        new_population.extend([child1, child2])
    return new_population


Explicação:

- random.randint: Define o ponto de corte no vetor binário.
- Segmentação: Divide os dois pais e troca os segmentos.
- Taxa de crossover: Controla a frequência de aplicação do crossover. Se não ocorrer crossover, os pais são mantidos.

## **Testando o Crossover**

In [ ]:
# Aplicando crossover na população selecionada
new_population = apply_crossover(selected_population, crossover_rate=0.7)

print("População após Crossover:")
for i, individual in enumerate(new_population):
    print(f"Indivíduo {i+1}: {individual}")


Saída Esperada:
Uma nova população de indivíduos (filhos), gerada com base na recombinação genética dos pais.

---